In [22]:
import pandas as pd

def extract_data_fromJSON(filename):
    data = json.load(open(filename))
    modified_data = []

    for row in data["data"]:
        doc = row["paragraphs"][0]
        for question in doc["qas"]:
            ele = {"context": doc["context"], "document_id": doc["document_id"]}
            ele["question"] = question["question"]
            ele["answers"] = {
                "text": [ans["text"] for ans in question["answers"]],
                "answer_start": [ans["answer_start"] for ans in question["answers"]],
            }  # question["answers"]
            ele["id"] = question["id"]
            modified_data.append(ele)
    return pd.DataFrame(modified_data)

In [23]:
test_json = 'covid-qa/covid-qa-test.json'
dev_json = 'covid-qa/covid-qa-dev.json'
train_json = 'covid-qa/covid-qa-train.json'

test_df = extract_data_fromJSON(test_json)
dev_df = extract_data_fromJSON(dev_json)
train_df = extract_data_fromJSON(train_json)

In [24]:
print(train_df.shape, dev_df.shape, test_df.shape)

(1417, 5) (203, 5) (375, 5)


In [25]:
test_df

,context,document_id,question,answers,id
0,Respiratory Viral Infections in Exacerbation o...,2504,When is this especially true?,{'text': ['when not all exacerbation events oc...,3963
1,Respiratory Viral Infections in Exacerbation o...,2504,What is the current understanding on viral-ind...,{'text': ['that viral infection increases airw...,3855
2,Respiratory Viral Infections in Exacerbation o...,2504,What have evoked new understandings as to the ...,{'text': ['Recent advances in in vitro air-liq...,3856
3,Respiratory Viral Infections in Exacerbation o...,2504,What is one of the major sources of exacerbat...,"{'text': ['Respiratory virus infection'], 'ans...",3854
4,Respiratory Viral Infections in Exacerbation o...,2504,What is the focus of this review?,{'text': [' recent novel findings that elucida...,3857
...,...,...,...,...,...
370,Obesity and risk of respiratory tract infectio...,1563,What conditions are considered upper respirato...,"{'text': ['common cold, pharyngitis, otitis, s...",328
371,Obesity and risk of respiratory tract infectio...,1563,What conditions are considered lower respirato...,"{'text': ['bronchitis, pneumonia and bronchiol...",329
372,Obesity and risk of respiratory tract infectio...,1563,What immune cells are primarily involved in el...,"{'text': ['NK cells (CD3-/CD16+/CD56+)'], 'ans...",330
373,Obesity and risk of respiratory tract infectio...,1563,What molecules have been shown to hinder T cel...,{'text': ['Excessive reactive oxygen species (...,331


PART - 1

In [27]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [28]:
model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)
